In [1]:
"""
Step1 引入需要的套件
"""
import os
import pandas as pd
import numpy as np
import openpyxl
from sklearn.ensemble import RandomForestRegressor

"""
Step3 建立模型
"""
# 隨機森林
M = RandomForestRegressor(max_depth=3, random_state=999, n_estimators=100, n_jobs=-1)



In [3]:
'''Step4 預測IC值'''
# 讀取 Excel 檔案
workbook = openpyxl.load_workbook("RF.xlsx")   # 如果檔案不存在，請先建立
sheet = workbook["預測IC"]    # 選擇工作表

"""
預測 bm 的IC
"""
df_x = pd.read_excel('bm.xlsx', sheet_name="bm補值")
df_y = pd.read_excel('bm.xlsx', sheet_name="bmIC")
df_x = df_x.T
df_y = df_y.T
df_x = df_x.iloc[2:,:]
df_y = df_y.iloc[2:,:]
n1,n2 = 178,312

for n in range(n1,n2):
    X = df_x.iloc[0:n,:].values
    Y = df_y.iloc[1:n+1,:].values.ravel()
    test_X = df_x.iloc[n:n+1,:].values
    M.fit(X, Y)
    predIC = M.predict(test_X)
    sheet.cell(row=3, column=n-n1+3, value=predIC[0])
    print(n)

"""
預測 size 的IC
"""
df_x = pd.read_excel('size.xlsx', sheet_name="size補值")
df_y = pd.read_excel('size.xlsx', sheet_name="sizeIC")
df_x = df_x.T
df_y = df_y.T
df_x = df_x.iloc[2:, :]
df_y = df_y.iloc[2:, :]
n1,n2 = 178,312

for n in range(n1,n2):
    X = df_x.iloc[0:n,:].values
    Y = df_y.iloc[1:n+1,:].values.ravel()
    test_X = df_x.iloc[n:n+1,:].values
    M.fit(X, Y)
    predIC = M.predict(test_X)
    sheet.cell(row=4, column=n-n1+3, value=predIC[0])
    print(n)

"""
預測 mom 的IC
"""
df_x = pd.read_excel('mom.xlsx', sheet_name="mom補值")
df_y = pd.read_excel('mom.xlsx', sheet_name="momIC")
df_x = df_x.T
df_y = df_y.T
df_x = df_x.iloc[2:, :]
df_y = df_y.iloc[2:, :]
n1,n2 = 166,300

for n in range(n1,n2):
    X = df_x.iloc[0:n,:].values
    Y = df_y.iloc[1:n+1,:].values.ravel()
    test_X = df_x.iloc[n:n+1,:].values
    M.fit(X, Y)
    predIC = M.predict(test_X)
    sheet.cell(row=5, column=n-n1+3, value=predIC[0])
    print(n)


"""
存檔
"""
workbook.save("RF.xlsx")

178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293


In [4]:
"""
Step5 匯入數據
"""
IC = pd.read_excel("RF.xlsx", sheet_name="預測IC").T
IC = np.array(IC.iloc[2:, 1:4])

#o讀取因數報酬資料
bm = pd.read_excel('bm.xlsx', sheet_name="bm補值").T
bm = np.array(bm.iloc[181:, :])
bm_ret = pd.read_excel('bm.xlsx', sheet_name="下個月月報酬補值").T
bm_ret = np.array(bm_ret.iloc[181:,:])

size = pd.read_excel('size.xlsx', sheet_name="size補值").T
size = np.array(size.iloc[181:, :])
size_ret = pd.read_excel('size.xlsx', sheet_name="下個月月報酬補值").T
size_ret = np.array(size_ret.iloc[181:,:])

mom = pd.read_excel('mom.xlsx', sheet_name="mom補值").T
mom = np.array(mom.iloc[169:, :])
mom_ret = pd.read_excel('mom.xlsx', sheet_name="下個月月報酬補值").T
mom_ret = np.array(mom_ret.iloc[169:,:])

In [5]:
"""
Step6 排名分組回測
"""
# 2013/12~2025/01
for n in range(134):
    c = pd.DataFrame()
    sel_index = np.argmax(abs(IC[n]))
    sel_value = IC[n][sel_index]

    if sel_index == 0:
        fac, sel, ret = bm[n], 'bm', bm_ret[n]
    elif sel_index == 1:
        fac, sel, ret = size[n], 'size', size_ret[n]
    elif sel_index == 2:
        fac, sel, ret = mom[n], 'mom', mom_ret[n]

    c['factor'] = fac
    c['return'] = ret
    c['rank'] = c['factor'].rank(method="max")
    c = c.sort_values(by="rank")

    row = [11,12,13,14,15,16,17,18,19,20,21,22,23,24]
    group = [185,92,37,18,10,9,8,7,6,5,4,3,2,1]

    for r, g in zip(row, group):
        high = np.mean(c['return'].iloc[-g:])
        low = np.mean(c['return'].iloc[:g])
        ans = high - low if sel_value > 0 else low - high
        sheet.cell(row=r, column=n+3, value=ans)
    print(n)

"""
存檔
"""
workbook.save("RF.xlsx")


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
